# M1M3 cell learning
Craig Lage - 14-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LightSource as LS
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable

from lsst_efd_client import EfdClient



In [ ]:
client = EfdClient('summit_efd')
FATABLE = M1M3FATable.FATABLE

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.subplot(1,1,1,aspect=1.0)
plt.xlabel("X position (m)")
plt.ylabel("Y position (m)")
plt.title("M1M3 Actuator positions and type\nHardpoints are approximate", fontsize=18)
types = [['SAA','NA', 'o', 'Z', 'b'], ['DAA','+Y', '^', '+Y','g'], ['DAA','-Y', 'v', '-Y', 'cyan'], \
         ['DAA','+X', '>', '+X', 'r'], ['DAA','-X', '<', '-X', 'r']]
for [type, orient, marker, label, color] in types: 
    xs = []
    ys = []
    for i in range(len(FATABLE)):
        x = FATABLE[i][M1M3FATable.FATABLE_XPOSITION]
        y = FATABLE[i][M1M3FATable.FATABLE_YPOSITION]
        if FATABLE[i][M1M3FATable.FATABLE_TYPE] == type and FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == orient:
            xs.append(x)
            ys.append(y)
        else:
            continue
    plt.scatter(xs, ys, marker=marker, color=color, s=200, label=label)        

# Now plot approximate hardpoint location
Rhp = 3.1 # Radius in meters
for i in range(6):
    theta = 2.0 * np.pi / 6.0 * float(i)
    if i == 0:
        plt.scatter(Rhp * np.cos(theta), Rhp * np.sin(theta), marker='o', color='magenta', s=200, label='HP')
    else:
        plt.scatter(Rhp * np.cos(theta), Rhp * np.sin(theta), marker='o', color='magenta', s=200, label='_nolegend_')
plt.legend(loc='lower left', fontsize=9)

In [ ]:
# I think this is going to work with more tweaking

lightsource = LS(azdeg=180, altdeg=78)
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(16,16))


greyColor = '0.9'
colors = []
xs = []
ys = []
for i in range(len(FATABLE)):
    x = FATABLE[i][M1M3FATable.FATABLE_XPOSITION]
    y = FATABLE[i][M1M3FATable.FATABLE_YPOSITION]
    xs.append(x)
    ys.append(y)
    if FATABLE[i][M1M3FATable.FATABLE_TYPE] == 'SAA':
        colors.append('blue'); colors.append('blue')
        colors.append(greyColor); colors.append(greyColor)
        colors.append(greyColor); colors.append(greyColor)
    else:
        if FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            colors.append('green'); colors.append('green')
            colors.append(greyColor); colors.append(greyColor)
            colors.append(greyColor); colors.append(greyColor)
        if FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] in ['+X', '-X']:
            colors.append('red'); colors.append('red')
            colors.append(greyColor); colors.append(greyColor)
            colors.append(greyColor); colors.append(greyColor)

zs = np.zeros([len(FATABLE)])
for i in range(len(FATABLE)):
    #zs[i] = 2.0 + 0.1 * xs[i]
    zs[i] = 2.0 + 0.25 * np.sqrt(xs[i]*xs[i] + ys[i]*ys[i])
    #zs[i] = 2.0 + np.random.rand()

dxs = 0.2 * np.ones([len(FATABLE)])
dys = 0.2 * np.ones([len(FATABLE)])
bottom = np.zeros([len(FATABLE)])
ax.bar3d(xs, ys, bottom, dxs, dys, zs, shade=True, alpha=0.5, lightsource=lightsource, color=colors)

ax.set_zlim(0, 4.0)
ax.view_init(elev=30., azim=225)


In [ ]:
fig = plt.figure(figsize=(8,8))
plt.subplot(1,1,1,aspect=1.0)
plt.xlabel("X position (m)")
plt.ylabel("Y position (m)")
plt.title("M1M3 Actuator positions", fontsize=18)

xs = []
ys = []
for i in range(len(FATABLE)):
    x = FATABLE[i][M1M3FATable.FATABLE_XPOSITION]
    y = FATABLE[i][M1M3FATable.FATABLE_YPOSITION]
    xs.append(x)
    ys.append(y)

zs = np.zeros([len(FATABLE)])
for i in range(len(FATABLE)):
    zs[i] = 2.0 + 0.1 * xs[i]

import matplotlib as mpl
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
plasma_big = mpl.colormaps['plasma']
newcmp = ListedColormap(plasma_big(np.linspace(0,4.0,256)))
plt.scatter(xs, ys, c=zs, cmap='plasma', s=200)

plt.colorbar(cmap=newcmp)

In [ ]:

fig = plt.figure(figsize=(8,8))
plt.subplot(1,1,1,aspect=1.0)
plt.xlabel("X position (m)")
plt.ylabel("Y position (m)")
plt.title("M1M3 Actuator positions", fontsize=18)

types = [['SAA','NA', 'o', 'Z'], ['DAA','+Y', '^', '+Y'], ['DAA','-Y', 'v', '-Y'], ['DAA','+X', '>', '+X'], ['DAA','-X', '<', '-X']]

for [type, orient, marker, label] in types: 
    xs = []
    ys = []
    zs = []
    for i in range(len(FATABLE)):
        x = FATABLE[i][M1M3FATable.FATABLE_XPOSITION]
        y = FATABLE[i][M1M3FATable.FATABLE_YPOSITION]
        if FATABLE[i][M1M3FATable.FATABLE_TYPE] == type and FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == orient:
            xs.append(x)
            ys.append(y)
            zs.append(2.0 + np.sqrt(x*x + y*y))
        else:
            continue
    plt.scatter(xs, ys, marker=marker, c=zs, cmap='RdBu_r', vmin=2.0, vmax=6.0, s=200, label=label)        
        
plt.colorbar(fraction=0.046, pad=0.04, cmap='RdBu_r')


In [ ]:
# Times to start looking at encoder values
start = Time("2023-04-05 16:35:00Z", scale='utc')
end = Time("2023-04-05 16:50:00Z", scale='utc')

In [ ]:
forces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", "*", start, end)

In [ ]:
len(forces)/(15*60)

In [ ]:
forces['timestamp'].head(5)

In [ ]:
forces['zForces27'].plot()

In [ ]:
forces['yForces83'].plot()

In [ ]:
FATABLE.

In [ ]:
for i in range(len(FATABLE)):
        if FATABLE[i][M1M3FATable.FATABLE_TYPE] == 'DAA' and FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == '+X':
            print(i, FATABLE[i][M1M3FATable.FATABLE_XINDEX])
for i in range(len(FATABLE)):
        if FATABLE[i][M1M3FATable.FATABLE_TYPE] == 'DAA' and FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == '-X':
            print(i, FATABLE[i][M1M3FATable.FATABLE_XINDEX])
for i in range(len(FATABLE)):
        if FATABLE[i][M1M3FATable.FATABLE_TYPE] == 'DAA' and FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == '+Y':
            print(i, FATABLE[i][M1M3FATable.FATABLE_YINDEX])

In [ ]:
count = 0
for i in range(len(FATABLE)):
        #if FATABLE[i][M1M3FATable.FATABLE_TYPE] == 'SAA':
        #    count += 1
        print(i, FATABLE[i][M1M3FATable.FATABLE_ZINDEX])


In [ ]:
len(forces)

In [ ]:
forces.iloc[25000]['zForces27']

In [ ]:
def barChartZ(df, ax, FATABLE, index):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_zlabel("Force (nt)")
    ax.set_title("M1M3 Actuator Z forces", fontsize=18)

    lightsource = LS(azdeg=180, altdeg=78)
    greyColor = '0.9'
    colors = []
    xs = []
    ys = []
    for i in range(len(FATABLE)):
        x = FATABLE[i][M1M3FATable.FATABLE_XPOSITION]
        y = FATABLE[i][M1M3FATable.FATABLE_YPOSITION]
        xs.append(x)
        ys.append(y)
        if FATABLE[i][M1M3FATable.FATABLE_TYPE] == 'SAA':
            colors.append('blue'); colors.append('blue')
            colors.append(greyColor); colors.append(greyColor)
            colors.append(greyColor); colors.append(greyColor)
        else:
            if FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
                colors.append('green'); colors.append('green')
                colors.append(greyColor); colors.append(greyColor)
                colors.append(greyColor); colors.append(greyColor)
            if FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] in ['+X', '-X']:
                colors.append('red'); colors.append('red')
                colors.append(greyColor); colors.append(greyColor)
                colors.append(greyColor); colors.append(greyColor)

    zs = np.zeros([len(FATABLE)])
    for i in range(len(FATABLE)):
        name=f"zForces{i}"
        zs[i] = df.iloc[index][name]

    dxs = 0.2 * np.ones([len(FATABLE)])
    dys = 0.2 * np.ones([len(FATABLE)])
    bottom = np.zeros([len(FATABLE)])
    ax.bar3d(xs, ys, bottom, dxs, dys, zs, shade=True, alpha=0.5, lightsource=lightsource, color=colors)

    ax.set_zlim(0, 1500)
    ax.view_init(elev=30., azim=225)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1, 1, 1, projection='3d')
barChartZ(forces, ax, FATABLE, 4000)

In [ ]:
def heatMapZ(df, ax, FATABLE, index):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M1M3 Actuator Z forces (nt)", fontsize=18)

    types = [['SAA','NA', 'o', 'Z'], ['DAA','+Y', '^', '+Y'], ['DAA','-Y', 'v', '-Y'], ['DAA','+X', '>', '+X'], ['DAA','-X', '<', '-X']]

    for [type, orient, marker, label] in types: 
        xs = []
        ys = []
        zs = []
        for i in range(len(FATABLE)):
            x = FATABLE[i][M1M3FATable.FATABLE_XPOSITION]
            y = FATABLE[i][M1M3FATable.FATABLE_YPOSITION]
            if FATABLE[i][M1M3FATable.FATABLE_TYPE] == type and FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == orient:
                xs.append(x)
                ys.append(y)
                name=f"zForces{i}"
                zs.append(df.iloc[index][name])
            else:
                continue
        im = ax.scatter(xs, ys, marker=marker, c=zs, cmap='RdBu_r', vmin=0.0, vmax=1500, s=200, label=label) 
    plt.colorbar(im, ax=ax,fraction=0.055, pad=0.02, cmap='RdBu_r')


In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1, 1, 1)

heatMapZ(forces, ax, FATABLE, 250)

In [ ]:
def lateralForces(df, ax, FATABLE, index, forceMax=1500):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M1M3 lateral forces (nt)", fontsize=18)
    ax.set_xlim(-4.5,4.5)
    ax.set_ylim(-4.5,4.5)
    types = [['DAA','+Y', '^', '+Y','g'], ['DAA','-Y', 'v', '-Y', 'cyan'], \
             ['DAA','+X', '>', '+X', 'r'], ['DAA','-X', '<', '-X', 'r']]
    for [type, orient, marker, label, color] in types: 
        xs = []
        ys = []
        arrowXs = []
        arrowYs = []
        for i in range(len(FATABLE)):
            x = FATABLE[i][M1M3FATable.FATABLE_XPOSITION]
            y = FATABLE[i][M1M3FATable.FATABLE_YPOSITION]
            if FATABLE[i][M1M3FATable.FATABLE_TYPE] == type and FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == orient:
                xs.append(x)
                ys.append(y)
                if orient == '+X':
                    name = f"xForces{FATABLE[i][M1M3FATable.FATABLE_XINDEX]}"
                    arrowXs.append(df.iloc[index][name] / forceMax)
                    arrowYs.append(0.0)
                if orient == '-X':
                    name = f"xForces{FATABLE[i][M1M3FATable.FATABLE_XINDEX]}"
                    arrowXs.append(-df.iloc[index][name] / forceMax)
                    arrowYs.append(0.0)
                if orient == '+Y':
                    name = f"yForces{FATABLE[i][M1M3FATable.FATABLE_YINDEX]}"
                    arrowXs.append(0.0)
                    arrowYs.append(df.iloc[index][name] / forceMax)
                if orient == '-Y':
                    name = f"yForces{FATABLE[i][M1M3FATable.FATABLE_YINDEX]}"
                    arrowXs.append(0.0)
                    arrowYs.append(-df.iloc[index][name] / forceMax)
            else:
                continue
        ax.scatter(xs, ys, marker=marker, color=color, s=50, label=label) 
        for ii in range(len(xs)):
            ax.arrow(xs[ii], ys[ii], arrowXs[ii], arrowYs[ii], color=color)

    ax.plot([-4.0,-3.0], [-4.0,-4.0], color='g')
    ax.text(-4.0, -4.3, f"{forceMax} nt")
            
        


In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1, 1, 1)

lateralForces(forces, ax, FATABLE, 250, forceMax=25)

In [ ]:
def actuatorLayout(ax, FATABLE):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M1M3 Actuator positions and type\nHardpoints are approximate", fontsize=18)
    types = [['SAA','NA', 'o', 'Z', 'b'], ['DAA','+Y', '^', '+Y','g'], ['DAA','-Y', 'v', '-Y', 'cyan'], \
             ['DAA','+X', '>', '+X', 'r'], ['DAA','-X', '<', '-X', 'r']]
    for [type, orient, marker, label, color] in types: 
        xs = []
        ys = []
        for i in range(len(FATABLE)):
            x = FATABLE[i][M1M3FATable.FATABLE_XPOSITION]
            y = FATABLE[i][M1M3FATable.FATABLE_YPOSITION]
            if FATABLE[i][M1M3FATable.FATABLE_TYPE] == type and FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == orient:
                xs.append(x)
                ys.append(y)
            else:
                continue
        ax.scatter(xs, ys, marker=marker, color=color, s=200, label=label)        

    # Now plot approximate hardpoint location
    Rhp = 3.1 # Radius in meters
    for i in range(6):
        theta = 2.0 * np.pi / 6.0 * float(i)
        if i == 0:
            ax.scatter(Rhp * np.cos(theta), Rhp * np.sin(theta), marker='o', color='magenta', s=200, label='HP')
        else:
            ax.scatter(Rhp * np.cos(theta), Rhp * np.sin(theta), marker='o', color='magenta', s=200, label='_nolegend_')
    ax.legend(loc='lower left', fontsize=9)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(1, 1, 1)

actuatorLayout(ax, FATABLE)

In [ ]:
def buildFakeSlewForces(df):
    slewEl = 30.0 * np.pi / 180.0
    slew = df.iloc[0:500].copy(deep=True)

    for n in range(500):
        if n <= 50:
            zforce = 1200.0
            yforce = 0.0
            xforce = 0.0
        elif n > 50 and n < 250:
            zforce = 1200.0 * np.cos(slewEl * (n - 50) / 400.0)
            yforce = 600.0 * np.sin(slewEl * (n - 50) / 400.0)
            xforce = 100.0 * (n - 50) / 200.0
        elif n >= 250 and n < 450:
            zforce = 1200.0 * np.cos(slewEl * (n - 50) / 400.0)
            yforce = 600.0 * np.sin(slewEl * (n - 50) / 400.0)
            xforce = 100.0 * (450 - n) / 200.0
        else:
            zforce = 1200.0 * np.cos(slewEl)
            yforce = 600.0 * np.sin(slewEl)
            xforce = 0.0
        for i in range(len(FATABLE)):
                name = f"zForces{FATABLE[i][M1M3FATable.FATABLE_ZINDEX]}"
                slew.iloc[n, slew.columns.get_loc(name)] = zforce
                if FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == '+Y':
                    name = f"yForces{FATABLE[i][M1M3FATable.FATABLE_YINDEX]}"
                    slew.iloc[n, slew.columns.get_loc(name)] = yforce
                if FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == '-X':
                    name = f"xForces{FATABLE[i][M1M3FATable.FATABLE_XINDEX]}"
                    slew.iloc[n, slew.columns.get_loc(name)] = xforce
    return slew


In [ ]:
slew = buildFakeSlewForces(forces)

In [ ]:
len(slew)

In [ ]:
name = f"zForces{FATABLE[27][M1M3FATable.FATABLE_ZINDEX]}"
print(slew.iloc[250][name])

In [ ]:
slew[name].plot()

In [ ]:
name = f"xForces{FATABLE[27][M1M3FATable.FATABLE_XINDEX]}"
print(slew.iloc[250][name])

In [ ]:
slew[name].plot()

In [ ]:
name = f"yForces{FATABLE[20][M1M3FATable.FATABLE_YINDEX]}"
print(slew.iloc[250][name])

In [ ]:
slew[name].plot()

In [ ]:
len(forces)

In [ ]:
# Times to start looking at encoder values
start = Time("2023-04-18T16:10:00", scale='tai')
end = Time("2023-04-18T16:15:00", scale='tai')

In [ ]:
forces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", "*", start.utc, end.utc)

In [ ]:
len(forces)/(5*60)

In [ ]:
forces['zForces27'].plot()

In [ ]:
forces['xForces9'].plot()

In [ ]:
fig = plt.figure(figsize=(16,16))
ax1 = fig.add_subplot(2,2,1)
actuatorLayout(ax1, FATABLE)
ax2 = fig.add_subplot(2,2,2, projection='3d')
barChartZ(forces, ax2, FATABLE, 250)
ax3 = fig.add_subplot(2,2,3)
lateralForces(forces, ax3, FATABLE, 250, forceMax=25)
ax4 = fig.add_subplot(2,2,4)
heatMapZ(forces, ax4, FATABLE, 250)

In [ ]:
plt.clf()
fig = plt.figure(figsize=(16,16))
ax1 = fig.add_subplot(2,2,1)
actuatorLayout(ax1, FATABLE)
ax2 = fig.add_subplot(2,2,2, projection='3d')
barChartZ(slew, ax2, FATABLE, 250)
ax3 = fig.add_subplot(2,2,3)
lateralForces(slew, ax3, FATABLE, 250, forceMax=300)
ax4 = fig.add_subplot(2,2,4)
heatMapZ(slew, ax4, FATABLE, 250)